In [0]:
df = spark.read.csv(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/employee.csv",
    header=True,
    inferSchema=True,
    sep="|",
    #timestampFormat="yyyy-MM-dd'T'HH:mm:ss.SSSXXX
)
df.display()

In [0]:
from pyspark.sql.functions import split
result_df = df.withColumn("skills",split("col_skills",","))

In [0]:
result_df.display()

In [0]:
result_df = result_df.withColumn("current_expected_salary",split("col_current_expected_salary",",")).cast("array<int>")

In [0]:
result_df.printSchema()
result_df.display()

In [0]:
result_df = result_df.drop("col_skills","col_current_expected_salary")
result_df.display()
result_df.write.mode("overwrite")

In [0]:
from pyspark.sql.functions import col

# Access the 'skills' array column, e.g., select the first skill for each row
result_df_skills = result_df.select(col("skills")[0].alias("first_skill"))
display(result_df_skills)

In [0]:
from pyspark.sql.functions import col

# result_df = result_df.withColumn("current_salary", col("current_expected_salary")[0])                    .withColumn("expected_salary", col("current_expected_salary")[1])


result_df_salaries = result_df.withColumn("expected_salary",result_df["current_expected_salary"][1])
result_df_salaries = result_df.withColumn("current_salary",result_df['current_expected_salary'][0])
result_df_salaries.withColumn("current_salary",col("current_salary").cast("int"))
result_df_salaries.withColumn("expected_salary",col("expected_salary").cast("int"))

In [0]:
result_df_salaries.printSchema()
result_df_salaries.display()

In [0]:
result_df = result_df.select(
    col("name"),
    col("current_expected_salary"),
    col("current_expected_salary")[0].alias("current_salary"),
    col("current_expected_salary")[1].alias("expected_salary"),
).filter(col("current_expected_salary")[0] > col("current_expected_salary")[1])
 
result_df.display()

In [0]:
from pyspark.sql.functions import array_contains,split
Pyspark_filtered_df = result_df.filter(
    array_contains(
        result_df.skills,  # Split string into array
        "PySpark"
    )
)
Pyspark_filtered_df.display()

In [0]:
Pyspark_filtered_df = Pyspark_filtered_df.withColumn("current_expected_salary",split("col_current_expected_salary",","))
result_df_salaries = Pyspark_filtered_df.withColumn("expected_salary",result_df["current_expected_salary"][1])
result_df_salaries = Pyspark_filtered_df.withColumn("current_salary",result_df['current_expected_salary'][0])

In [0]:
Pyspark_filtered_df.printSchema()
Pyspark_filtered_df.display()

In [0]:
from pyspark.sql.functions import explode
result_df.select(explode("skills").alias("words")).groupBy("words").count().display()